In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f09642401210da2948158c4af8382bf9b6284b0da4eaabb4d3069a0d33f998d7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, count, col
import matplotlib.pyplot as plt
from pyspark.sql.functions import to_date, date_format
import pandas as pd
import numpy as np
from scipy import stats

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.format("csv").option("header", True).load("/content/AIT664-Group001-Dataset01-Connecticut-RealEstateData.csv")
print(df.dtypes)
df.printSchema()
S_df = spark.read.format("csv").option("header", True).load("/content/School_Updated(2).csv")
print(S_df.dtypes)
S_df.printSchema()
H_df = spark.read.format("csv").option("header", True).load("/content/Health(2).csv")
print(H_df.dtypes)
H_df.printSchema()
C_df = spark.read.format("csv").option("header", True).load("/content/Crime_Updated.csv")
print(C_df.dtypes)
C_df.printSchema()

[('Town', 'string'), ('Residential Type', 'string'), ('Serial Number', 'string'), ('List Year', 'string'), ('Date Recorded', 'string'), ('Address', 'string'), ('Assessed Value', 'string'), ('Sale Amount', 'string'), ('Sales Ratio', 'string'), ('Location', 'string')]
root
 |-- Town: string (nullable = true)
 |-- Residential Type: string (nullable = true)
 |-- Serial Number: string (nullable = true)
 |-- List Year: string (nullable = true)
 |-- Date Recorded: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Assessed Value: string (nullable = true)
 |-- Sale Amount: string (nullable = true)
 |-- Sales Ratio: string (nullable = true)
 |-- Location: string (nullable = true)

[('School ID', 'string'), ('District Name', 'string'), ('School Name', 'string'), ('Organization Type', 'string'), ('Organization Code', 'string'), ('Address', 'string'), ('Town', 'string'), ('Zipcode', 'string'), ('Phone', 'string'), ('State', 'string'), ('geometry', 'string'), ('Student_Open_Date',

In [ ]:
# Convert Year column to integer type
df = df.withColumn("List Year", df["List Year"].cast("integer"))
# Convert Date Recorded column to date format
df = df.withColumn("Date Recorded", to_date(df["Date Recorded"], "M/d/yyyy"))
df = df.withColumn("Date Recorded", date_format("Date Recorded", "MM/dd/yyyy"))
# Convert Assessed Value column to floating point number
df = df.withColumn("Assessed Value", df["Assessed Value"].cast("double"))
# Convert Sale Amount column to floating point number
df = df.withColumn("Sale Amount", df["Sale Amount"].cast("double"))
# Convert Sales Ratio column to floating point number
df = df.withColumn("Sales Ratio", df["Sales Ratio"].cast("double "))
print(df.dtypes)
df.printSchema()

[('Town', 'string'), ('Residential Type', 'string'), ('Serial Number', 'string'), ('List Year', 'int'), ('Date Recorded', 'string'), ('Address', 'string'), ('Assessed Value', 'double'), ('Sale Amount', 'double'), ('Sales Ratio', 'double'), ('Location', 'string')]
root
 |-- Town: string (nullable = true)
 |-- Residential Type: string (nullable = true)
 |-- Serial Number: string (nullable = true)
 |-- List Year: integer (nullable = true)
 |-- Date Recorded: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Assessed Value: double (nullable = true)
 |-- Sale Amount: double (nullable = true)
 |-- Sales Ratio: double (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
# Convert "Date Recorded" column to date format
df = df.withColumn("Date Recorded", to_date(df["Date Recorded"], "M/d/yyyy"))
# Print data types and schema for verification
print(df.dtypes)
df.printSchema()

[('Town', 'string'), ('Residential Type', 'string'), ('Serial Number', 'string'), ('List Year', 'int'), ('Date Recorded', 'date'), ('Address', 'string'), ('Assessed Value', 'double'), ('Sale Amount', 'double'), ('Sales Ratio', 'double'), ('Location', 'string')]
root
 |-- Town: string (nullable = true)
 |-- Residential Type: string (nullable = true)
 |-- Serial Number: string (nullable = true)
 |-- List Year: integer (nullable = true)
 |-- Date Recorded: date (nullable = true)
 |-- Address: string (nullable = true)
 |-- Assessed Value: double (nullable = true)
 |-- Sale Amount: double (nullable = true)
 |-- Sales Ratio: double (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
df = df.withColumn("Year", year(col("Date Recorded")))

In [ ]:
year_wise_sales = df.groupBy("Year").agg(count("*").alias("Total Houses Sold"))
year_wise_sales = year_wise_sales.orderBy("Year")
year_wise_sales.show()

+----+-----------------+
|Year|Total Houses Sold|
+----+-----------------+
|2001|                3|
|2003|                1|
|2004|                4|
|2005|                1|
|2006|             6182|
|2007|            20833|
|2008|             7119|
|2009|             7706|
|2010|             8193|
|2011|             6993|
|2012|             6968|
|2013|             7216|
|2014|             8142|
|2015|             9844|
|2016|             9684|
|2017|             9404|
|2018|             9100|
|2019|             9778|
|2020|            24009|
|2021|            55509|
+----+-----------------+
only showing top 20 rows



In [ ]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
# Group by year and house type, then count houses for each type
df = df.filter(df['Year'].isin(years))
house_type_sales = df.groupBy("Year", "Residential Type").agg(count("*").alias("Houses Sold")).orderBy("Year")
house_type_sales_df = house_type_sales.toPandas()
desired_types = ["Condo", "Single Family", "Two Family", "Three Family", "Four Family"]
pivot_table = house_type_sales_df.pivot_table(values="Houses Sold", index="Year", columns=["Residential Type"]).reindex(columns=desired_types).reset_index()
print(pivot_table)

Residential Type  Year  Condo  Single Family  Two Family  Three Family  \
0                 2010   1441           5932         492           289   
1                 2011   1219           5104         406           218   
2                 2012   1145           5251         365           170   
3                 2013   1187           5519         311           170   
4                 2014   1376           6103         403           224   
5                 2015   1640           7446         500           220   
6                 2016   1677           7283         460           213   
7                 2017   1630           7043         464           221   

Residential Type  Four Family  
0                          39  
1                          46  
2                          37  
3                          29  
4                          36  
5                          38  
6                          51  
7                          46  


In [ ]:
house_types = ["Condo", "Single Family"]
df = df[df['Residential Type'].isin(house_types)]
house_type_sales_town = df.groupBy("Town", "Residential Type").agg(count("*").alias("Houses Sold")).orderBy(["Houses Sold"])
house_type_sales_town_df = house_type_sales_town.toPandas()
house_type_sales_town_df

,Town,Residential Type,Houses Sold
0,Morris,Condo,1
1,Chaplin,Condo,1
2,Goshen,Condo,1
3,Bethlehem,Condo,1
4,Weston,Condo,1
...,...,...,...
306,Greenwich,Single Family,886
307,West Hartford,Single Family,931
308,Stamford,Single Family,1117
309,Norwalk,Single Family,1178


In [ ]:
condo_df = house_type_sales_town_df[house_type_sales_town_df['Residential Type'] == 'Condo']
single_family_df = house_type_sales_town_df[house_type_sales_town_df['Residential Type'] == 'Single Family']

In [ ]:
house_type_sales_town_year = df.groupBy("Year", "Town", "Residential Type").agg(count("*").alias("Houses Sold")).orderBy(["Year","Town"])
house_type_sales_town_year_df = house_type_sales_town_year.toPandas()
house_type_sales_town_year_df

,Year,Town,Residential Type,Houses Sold
0,2010,Andover,Single Family,9
1,2010,Ansonia,Condo,1
2,2010,Ansonia,Single Family,20
3,2010,Ashford,Single Family,12
4,2010,Ashford,Condo,1
...,...,...,...,...
2239,2017,Wolcott,Single Family,59
2240,2017,Woodbridge,Single Family,21
2241,2017,Woodbury,Condo,14
2242,2017,Woodbury,Single Family,31


In [ ]:
house_types = ["Condo", "Single Family"]
pivot_table = house_type_sales_town_year_df.pivot_table(values="Houses Sold", index=["Year","Town"], columns=["Residential Type"]).reindex(columns=house_types).reset_index()
print(pivot_table)

Residential Type  Year           Town  Condo  Single Family
0                 2010        Andover    NaN            9.0
1                 2010        Ansonia    1.0           20.0
2                 2010        Ashford    1.0           12.0
3                 2010           Avon   30.0           47.0
4                 2010    Barkhamsted    NaN            7.0
...                ...            ...    ...            ...
1339              2017  Windsor Locks   12.0           32.0
1340              2017        Wolcott    5.0           59.0
1341              2017     Woodbridge    NaN           21.0
1342              2017       Woodbury   14.0           31.0
1343              2017      Woodstock    NaN            6.0

[1344 rows x 4 columns]


In [ ]:
# prompt: Using dataframe pivot_table: replace NaN with 0

pivot_table.fillna(0, inplace=True)


In [ ]:
print(pivot_table)

Residential Type  Year           Town  Condo  Single Family
0                 2010        Andover    0.0            9.0
1                 2010        Ansonia    1.0           20.0
2                 2010        Ashford    1.0           12.0
3                 2010           Avon   30.0           47.0
4                 2010    Barkhamsted    0.0            7.0
...                ...            ...    ...            ...
1339              2017  Windsor Locks   12.0           32.0
1340              2017        Wolcott    5.0           59.0
1341              2017     Woodbridge    0.0           21.0
1342              2017       Woodbury   14.0           31.0
1343              2017      Woodstock    0.0            6.0

[1344 rows x 4 columns]


In [ ]:
df.printSchema()
S_df.printSchema()

root
 |-- Town: string (nullable = true)
 |-- Residential Type: string (nullable = true)
 |-- Serial Number: string (nullable = true)
 |-- List Year: integer (nullable = true)
 |-- Date Recorded: date (nullable = true)
 |-- Address: string (nullable = true)
 |-- Assessed Value: double (nullable = true)
 |-- Sale Amount: double (nullable = true)
 |-- Sales Ratio: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Year: integer (nullable = true)

root
 |-- School ID: string (nullable = true)
 |-- District Name: string (nullable = true)
 |-- School Name: string (nullable = true)
 |-- Organization Type: string (nullable = true)
 |-- Organization Code: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Town: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- State: string (nullable = true)
 |-- geometry: string (nullable = true)
 |-- Student_Open_Date: string (nullable = true)
 |-- Open_Year: string 

In [ ]:
# prompt: rename the column city with Town

H_df = H_df.withColumnRenamed("City", "Town")
H_df.printSchema()


root
 |-- Credential ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Business Name: string (nullable = true)
 |-- DBA: string (nullable = true)
 |-- Full Credential Code: string (nullable = true)
 |-- Credential: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Effective Date: string (nullable = true)
 |-- Expiration Date: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Town: string (nullable = true)
 |-- Zip Code: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Healthcare ID: string (nullable = true)
 |-- Issue Year: string (nullable = true)
 |-- Effective Year: string (nullable = true)
 |-- Expiration Year: string (nullable = true)



In [ ]:
from pyspark.sql.functions import initcap

# Capitalize the first letter of each word in the "Town" column
H_df = H_df.withColumn("Town", initcap(H_df["Town"]))

# Show the DataFrame
H_df.show()

+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+--------------+---------------+--------------------+--------------+--------+--------------------+-------------+----------+--------------+---------------+
|Credential ID|                Name|                Type|       Business Name|                 DBA|Full Credential Code|          Credential|           Status|Issue Date|Effective Date|Expiration Date|             Address|          Town|Zip Code|            Location|Healthcare ID|Issue Year|Effective Year|Expiration Year|
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+--------------+---------------+--------------------+--------------+--------+--------------------+-------------+----------+--------------+---------------+
|      1003908|  ATRIA LARSO

In [ ]:
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg
# Group the data by 'Town'
town_grouped = C_df.groupBy('Town')

# Aggregate by town and year
aggregated_data_town_year = C_df.groupBy("Town", "Year") \
    .agg(spark_sum("Number of Crimes Committed").alias("Total Crimes Committed"),
         spark_avg("Number of Crimes Committed per 100,000").alias("Average Crimes per 100,000"))

# Show the aggregated data
print("Aggregated data by town and year:")
aggregated_data_town_year.show()

Aggregated data by town and year:
+-----------+----+----------------------+--------------------------+
|       Town|Year|Total Crimes Committed|Average Crimes per 100,000|
+-----------+----+----------------------+--------------------------+
|    Chester|2012|                  54.0|        124.44545454545455|
|  Naugatuck|2010|                1997.0|         558.2181818181818|
|    Shelton|2013|                1392.0|         314.6636363636364|
|   Branford|2010|                2545.0|         768.5909090909089|
|   Coventry|2015|                 279.0|        209.27272727272728|
|  Fairfield|2012|                4009.0|         602.8818181818182|
|   Franklin|2010|                 138.0|         651.3818181818182|
|  Harwinton|2013|                 102.0|        166.34545454545457|
|Middlefield|2010|                 138.0|        284.40909090909093|
| Burlington|2013|                 129.0|        135.67272727272726|
| Burlington|2016|                  48.0|        50.690909090909095|


In [ ]:
df.show()

+------------+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----+
|        Town|Residential Type|Serial Number|List Year|Date Recorded|            Address|Assessed Value|Sale Amount|Sales Ratio|            Location|Year|
+------------+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----+
|        Avon|           Condo|        90146|     2009|   2010-04-12|      3 WILLOW LANE|      125870.0|   168500.0|       0.75|POINT (-72.875838...|2010|
|     Windsor|           Condo|        90270|     2009|   2010-06-18|     91 VICTORIA ST|       52990.0|    87000.0|       0.61|POINT (-72.657879...|2010|
|  Rocky Hill|           Condo|       100183|     2010|   2011-08-11|    27 HICKORY LANE|      141540.0|   168500.0|       0.84|POINT (-73.451804...|2011|
|   Stratford|           Condo|       100354|     2010|   2011-04-29| 

In [ ]:
joined_df = df.join(aggregated_data_town_year, ['Town', 'Year'], 'left')

# Show the joined DataFrame
print("Joined DataFrame:")
joined_df.show()

Joined DataFrame:
+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+
|        Town|Year|Residential Type|Serial Number|List Year|Date Recorded|            Address|Assessed Value|Sale Amount|Sales Ratio|            Location|Total Crimes Committed|Average Crimes per 100,000|
+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+
|        Avon|2010|           Condo|        90146|     2009|   2010-04-12|      3 WILLOW LANE|      125870.0|   168500.0|       0.75|POINT (-72.875838...|                 577.0|        315.72727272727263|
|     Windsor|2010|           Condo|        90270|     2009|   2010-06-18|     91 VICTORIA ST|       52990.0|    87000.0|       0.61|POINT (-72.657879...|        

In [ ]:
df = joined_df
df_pandas = df.toPandas()
df_pandas

,Town,Year,Residential Type,Serial Number,List Year,Date Recorded,Address,Assessed Value,Sale Amount,Sales Ratio,Location,Total Crimes Committed,"Average Crimes per 100,000"
0,Avon,2010,Condo,90146,2009,2010-04-12,3 WILLOW LANE,125870.0,168500.0,0.75,POINT (-72.8758384 41.7696552),577.0,315.727273
1,Windsor,2010,Condo,90270,2009,2010-06-18,91 VICTORIA ST,52990.0,87000.0,0.61,POINT (-72.65787947200603 41.87221605797811),1596.0,489.609091
2,Rocky Hill,2011,Condo,100183,2010,2011-08-11,27 HICKORY LANE,141540.0,168500.0,0.84,POINT (-73.45180420689245 41.2790051414073),816.0,392.709091
3,Stratford,2011,Condo,100354,2010,2011-04-29,6 SHORELINE DR,451500.0,420000.0,1.08,POINT (-73.12735727422778 41.14845306094261),5140.0,889.290909
4,Berlin,2012,Condo,110102,2011,2012-02-10,7 QUARRY LN,250000.0,351450.0,0.71,POINT (-73.4111588 41.154743),995.0,471.018182
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60991,Putnam,2017,Condo,170049,2017,2017-12-06,63 PERRY ST UNIT 107,88600.0,116700.0,0.76,POINT (-71.8961 41.91473),574.0,550.681818
60992,Westport,2017,Single Family,170022,2017,2017-10-17,6 WHEELER GATE,1124500.0,1720000.0,0.65,POINT (-73.36249 41.13179),912.0,305.872727
60993,Wolcott,2017,Single Family,1700068,2017,2017-12-27,167 LYMAN RD,223270.0,305000.0,0.73,POINT (-72.9943593 41.6020151),849.0,481.936364
60994,Watertown,2017,Single Family,17301,2017,2017-07-17,46 MT FAIR DR,207400.0,322500.0,0.64,POINT (-73.10077 41.61491),996.0,397.127273


In [ ]:
S_df.show()

+---------+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-------+------------+-----+--------------------+-----------------+---------+
|School ID|       District Name|         School Name|Organization Type|Organization Code|             Address|         Town|Zipcode|       Phone|State|            geometry|Student_Open_Date|Open_Year|
+---------+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-------+------------+-----+--------------------+-----------------+---------+
|        0|Vernon School Dis...| Skinner Road School|   Public Schools|          1461011|      90 Skinner Rd.|       Vernon|   6066|860-870-6180|   CT|POINT (-72.491502...|       1984-01-07|     1984|
|        2|Norwalk School Di...|West Rocks Middle...|   Public Schools|          1035411|  81 West Rocks Road|      Norwalk|   6851|203-899-2970|   CT|POINT (-73.416113...|       1984-01-07|     1

In [ ]:
# Filter data for schools opened on or before 2017
schools_until_2017 = S_df.filter(col('Open_Year') <= '2017')

# Group by 'Town' and count the number of unique schools
schools_count_per_town = schools_until_2017.groupBy('Town').agg({'School Name': 'count'}).withColumnRenamed('count(School Name)', 'Number of Schools')

schools_count_per_town.show()

+-------------------+-----------------+
|               Town|Number of Schools|
+-------------------+-----------------+
|         Plainfield|                3|
|      Old Greenwich|                1|
|   Stafford Springs|                4|
|             Monroe|                5|
|         Brookfield|                4|
|North Grosvenordale|                3|
|           Sterling|                1|
|             Bolton|                2|
|      Windsor Locks|                4|
|            Wolcott|                5|
|         Woodbridge|                1|
|           Cromwell|                4|
|     North Franklin|                1|
|        New Milford|                5|
|           Guilford|                7|
|         Manchester|                7|
|          Thomaston|                3|
|           Dayville|                2|
|             Wilton|                4|
|          Stratford|               12|
+-------------------+-----------------+
only showing top 20 rows



In [ ]:
df = schools_count_per_town
df_pandas = schools_count_per_town.toPandas()
df_pandas

,Town,Number of Schools
0,Plainfield,3
1,Old Greenwich,1
2,Stafford Springs,4
3,Monroe,5
4,Brookfield,4
...,...,...
163,Bethany,1
164,Unionville,1
165,Preston,1
166,Marlborough,1


In [ ]:
# Join 'joined_df' with 'schools_count_per_town' on the 'Town' column
joined_result = joined_df.join(schools_count_per_town, ['Town'], 'left')

# Show the result
joined_result.show()

+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+
|        Town|Year|Residential Type|Serial Number|List Year|Date Recorded|            Address|Assessed Value|Sale Amount|Sales Ratio|            Location|Total Crimes Committed|Average Crimes per 100,000|Number of Schools|
+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+
|        Avon|2010|           Condo|        90146|     2009|   2010-04-12|      3 WILLOW LANE|      125870.0|   168500.0|       0.75|POINT (-72.875838...|                 577.0|        315.72727272727263|                5|
|     Windsor|2010|           Condo|        90270|     2009|   2010-06-18|     91 VICTORIA ST|       52990.0

In [ ]:
df = joined_result
df_pandas = joined_result.toPandas()
df_pandas

,Town,Year,Residential Type,Serial Number,List Year,Date Recorded,Address,Assessed Value,Sale Amount,Sales Ratio,Location,Total Crimes Committed,"Average Crimes per 100,000",Number of Schools
0,Avon,2010,Condo,90146,2009,2010-04-12,3 WILLOW LANE,125870.0,168500.0,0.75,POINT (-72.8758384 41.7696552),577.0,315.727273,5.0
1,Windsor,2010,Condo,90270,2009,2010-06-18,91 VICTORIA ST,52990.0,87000.0,0.61,POINT (-72.65787947200603 41.87221605797811),1596.0,489.609091,5.0
2,Rocky Hill,2011,Condo,100183,2010,2011-08-11,27 HICKORY LANE,141540.0,168500.0,0.84,POINT (-73.45180420689245 41.2790051414073),816.0,392.709091,4.0
3,Stratford,2011,Condo,100354,2010,2011-04-29,6 SHORELINE DR,451500.0,420000.0,1.08,POINT (-73.12735727422778 41.14845306094261),5140.0,889.290909,12.0
4,Berlin,2012,Condo,110102,2011,2012-02-10,7 QUARRY LN,250000.0,351450.0,0.71,POINT (-73.4111588 41.154743),995.0,471.018182,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60991,Putnam,2017,Condo,170049,2017,2017-12-06,63 PERRY ST UNIT 107,88600.0,116700.0,0.76,POINT (-71.8961 41.91473),574.0,550.681818,3.0
60992,Westport,2017,Single Family,170022,2017,2017-10-17,6 WHEELER GATE,1124500.0,1720000.0,0.65,POINT (-73.36249 41.13179),912.0,305.872727,7.0
60993,Wolcott,2017,Single Family,1700068,2017,2017-12-27,167 LYMAN RD,223270.0,305000.0,0.73,POINT (-72.9943593 41.6020151),849.0,481.936364,5.0
60994,Watertown,2017,Single Family,17301,2017,2017-07-17,46 MT FAIR DR,207400.0,322500.0,0.64,POINT (-73.10077 41.61491),996.0,397.127273,2.0


In [ ]:
from pyspark.sql.functions import col, round

# Round the 'Average Crimes per 100,000' column to 2 decimal places
joined_result = joined_result.withColumn('Average Crimes per 100,000', round(col('Average Crimes per 100,000'), 2))

# Show the modified DataFrame
joined_result.show()

+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+
|        Town|Year|Residential Type|Serial Number|List Year|Date Recorded|            Address|Assessed Value|Sale Amount|Sales Ratio|            Location|Total Crimes Committed|Average Crimes per 100,000|Number of Schools|
+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+
|        Avon|2010|           Condo|        90146|     2009|   2010-04-12|      3 WILLOW LANE|      125870.0|   168500.0|       0.75|POINT (-72.875838...|                 577.0|                    315.73|                5|
|     Windsor|2010|           Condo|        90270|     2009|   2010-06-18|     91 VICTORIA ST|       52990.0

In [ ]:
H_df.printSchema()
H_df.show()

root
 |-- Credential ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Business Name: string (nullable = true)
 |-- DBA: string (nullable = true)
 |-- Full Credential Code: string (nullable = true)
 |-- Credential: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Effective Date: string (nullable = true)
 |-- Expiration Date: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Town: string (nullable = true)
 |-- Zip Code: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Healthcare ID: string (nullable = true)
 |-- Issue Year: string (nullable = true)
 |-- Effective Year: string (nullable = true)
 |-- Expiration Year: string (nullable = true)

+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------+--------------+----------

In [ ]:
# Filter data for facilities issued on or before 2017
facilities_until_2017 = H_df.filter(col('Issue Year') <= 2017)

# Group by 'Town' and count the number of unique facilities
facilities_count_per_town = facilities_until_2017.groupBy('Town').agg({'Credential ID': 'count'}).withColumnRenamed('count(Credential ID)', 'Number of Facilities')

facilities_count_per_town.show()

+----------------+--------------------+
|            Town|Number of Facilities|
+----------------+--------------------+
|      Plainfield|                   1|
|Stafford Springs|                   2|
|      Brookfield|                   1|
|   Windsor Locks|                   1|
|      Woodbridge|                   3|
|         Wolcott|                   1|
|        Cromwell|                   4|
|     New Milford|                   2|
|        Guilford|                   6|
|      Manchester|                   3|
|        Dayville|                   1|
|          Wilton|                   5|
|       Stratford|                   4|
|          Orange|                   2|
|       Newington|                   4|
|        Suffield|                   2|
|      Kensington|                   1|
|Vernon Rockville|                   1|
|       Fairfield|                   5|
|       Westbrook|                   1|
+----------------+--------------------+
only showing top 20 rows



In [ ]:
# Perform left join between 'joined_result' and 'facilities_count_per_town' on 'Town' column
final_result = joined_result.join(facilities_count_per_town, ['Town'], 'left')

# Show the final merged result
final_result.show()

+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+--------------------+
|        Town|Year|Residential Type|Serial Number|List Year|Date Recorded|            Address|Assessed Value|Sale Amount|Sales Ratio|            Location|Total Crimes Committed|Average Crimes per 100,000|Number of Schools|Number of Facilities|
+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+--------------------+
|        Avon|2010|           Condo|        90146|     2009|   2010-04-12|      3 WILLOW LANE|      125870.0|   168500.0|       0.75|POINT (-72.875838...|                 577.0|                    315.73|                5|                   4|
|     Windsor|2010|     

In [ ]:
# Rename the 'Number of Facilities' column to 'Number of Healthcare facilities'
final_result = final_result.withColumnRenamed("Number of Facilities", "Number of Healthcare facilities")

# Show the final result
final_result.show()

+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+-------------------------------+
|        Town|Year|Residential Type|Serial Number|List Year|Date Recorded|            Address|Assessed Value|Sale Amount|Sales Ratio|            Location|Total Crimes Committed|Average Crimes per 100,000|Number of Schools|Number of Healthcare facilities|
+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+-------------------------------+
|        Avon|2010|           Condo|        90146|     2009|   2010-04-12|      3 WILLOW LANE|      125870.0|   168500.0|       0.75|POINT (-72.875838...|                 577.0|                    315.73|                5|             

In [35]:
# Iterate over each column in the final_result DataFrame
for col in final_result.columns:
    # Count the null values and empty strings in the column
    null_count = final_result.filter(final_result[col].isNull() | (final_result[col] == "")).count()
    # Print the column name and corresponding null count
    print(f"Column: {col}\t Null Count: {null_count}")

# Count the total number of rows in the final_result DataFrame
total_count = final_result.count()
# Print the total count of rows
print(f"Total Count: {total_count}")

Column: Town	 Null Count: 0
Column: Year	 Null Count: 0
Column: Residential Type	 Null Count: 0
Column: Serial Number	 Null Count: 0
Column: List Year	 Null Count: 0
Column: Date Recorded	 Null Count: 0
Column: Address	 Null Count: 0
Column: Assessed Value	 Null Count: 0
Column: Sale Amount	 Null Count: 0
Column: Sales Ratio	 Null Count: 6
Column: Location	 Null Count: 4
Column: Total Crimes Committed	 Null Count: 0
Column: Average Crimes per 100,000	 Null Count: 0
Column: Number of Schools	 Null Count: 6010
Column: Number of Healthcare facilities	 Null Count: 13557
Total Count: 60996


In [36]:
# Replace null values in "Number of Schools" and "Number of Healthcare Facilities" columns with zero
final_result = final_result.fillna({'Number of Schools': 0, 'Number of Healthcare facilities': 0})

# Display the updated DataFrame
final_result.show()

+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+-------------------------------+
|        Town|Year|Residential Type|Serial Number|List Year|Date Recorded|            Address|Assessed Value|Sale Amount|Sales Ratio|            Location|Total Crimes Committed|Average Crimes per 100,000|Number of Schools|Number of Healthcare facilities|
+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+-------------------------------+
|        Avon|2010|           Condo|        90146|     2009|   2010-04-12|      3 WILLOW LANE|      125870.0|   168500.0|       0.75|POINT (-72.875838...|                 577.0|                    315.73|                5|             

In [37]:
# Iterate over each column in the final_result DataFrame
for col in final_result.columns:
    # Count the null values and empty strings in the column
    null_count = final_result.filter(final_result[col].isNull() | (final_result[col] == "")).count()
    # Print the column name and corresponding null count
    print(f"Column: {col}\t Null Count: {null_count}")

# Count the total number of rows in the final_result DataFrame
total_count = final_result.count()
# Print the total count of rows
print(f"Total Count: {total_count}")

Column: Town	 Null Count: 0
Column: Year	 Null Count: 0
Column: Residential Type	 Null Count: 0
Column: Serial Number	 Null Count: 0
Column: List Year	 Null Count: 0
Column: Date Recorded	 Null Count: 0
Column: Address	 Null Count: 0
Column: Assessed Value	 Null Count: 0
Column: Sale Amount	 Null Count: 0
Column: Sales Ratio	 Null Count: 6
Column: Location	 Null Count: 4
Column: Total Crimes Committed	 Null Count: 0
Column: Average Crimes per 100,000	 Null Count: 0
Column: Number of Schools	 Null Count: 0
Column: Number of Healthcare facilities	 Null Count: 0
Total Count: 60996


In [38]:
# Show rows where "Sales Ratio" or "Location" is null
final_result.filter((final_result['Sales Ratio'].isNull()) | (final_result['Location'].isNull())).show()


+----------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+-------------------------------+
|      Town|Year|Residential Type|Serial Number|List Year|Date Recorded|            Address|Assessed Value|Sale Amount|Sales Ratio|            Location|Total Crimes Committed|Average Crimes per 100,000|Number of Schools|Number of Healthcare facilities|
+----------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+-------------------------------+
|    Berlin|2010|   Single Family|       900243|     2009|   2010-09-13|      405 ALLING ST|      177400.0|   125500.0|       1.41|                NULL|                1083.0|                    515.01|                3|                     

In [39]:
# Remove rows where "Location" is null
final_result = final_result.filter(final_result['Location'].isNotNull())

# Show the cleaned DataFrame
final_result.show()

+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+-------------------------------+
|        Town|Year|Residential Type|Serial Number|List Year|Date Recorded|            Address|Assessed Value|Sale Amount|Sales Ratio|            Location|Total Crimes Committed|Average Crimes per 100,000|Number of Schools|Number of Healthcare facilities|
+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+-------------------------------+
|        Avon|2010|           Condo|        90146|     2009|   2010-04-12|      3 WILLOW LANE|      125870.0|   168500.0|       0.75|POINT (-72.875838...|                 577.0|                    315.73|                5|             

In [40]:
# Replace null values in "Sales Ratio" column with 0
final_result = final_result.fillna(0, subset=['Sales Ratio'])

# Show the DataFrame after replacing null values in "Sales Ratio" column
final_result.show()

+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+-------------------------------+
|        Town|Year|Residential Type|Serial Number|List Year|Date Recorded|            Address|Assessed Value|Sale Amount|Sales Ratio|            Location|Total Crimes Committed|Average Crimes per 100,000|Number of Schools|Number of Healthcare facilities|
+------------+----+----------------+-------------+---------+-------------+-------------------+--------------+-----------+-----------+--------------------+----------------------+--------------------------+-----------------+-------------------------------+
|        Avon|2010|           Condo|        90146|     2009|   2010-04-12|      3 WILLOW LANE|      125870.0|   168500.0|       0.75|POINT (-72.875838...|                 577.0|                    315.73|                5|             

In [41]:
# Iterate over each column in the final_result DataFrame
for col in final_result.columns:
    # Count the null values and empty strings in the column
    null_count = final_result.filter(final_result[col].isNull() | (final_result[col] == "")).count()
    # Print the column name and corresponding null count
    print(f"Column: {col}\t Null Count: {null_count}")

# Count the total number of rows in the final_result DataFrame
total_count = final_result.count()
# Print the total count of rows
print(f"Total Count: {total_count}")

Column: Town	 Null Count: 0
Column: Year	 Null Count: 0
Column: Residential Type	 Null Count: 0
Column: Serial Number	 Null Count: 0
Column: List Year	 Null Count: 0
Column: Date Recorded	 Null Count: 0
Column: Address	 Null Count: 0
Column: Assessed Value	 Null Count: 0
Column: Sale Amount	 Null Count: 0
Column: Sales Ratio	 Null Count: 0
Column: Location	 Null Count: 0
Column: Total Crimes Committed	 Null Count: 0
Column: Average Crimes per 100,000	 Null Count: 0
Column: Number of Schools	 Null Count: 0
Column: Number of Healthcare facilities	 Null Count: 0
Total Count: 60992


In [ ]:
# Repartition the DataFrame to have only one partition
final_result = final_result.repartition(1)

# Write the DataFrame to a new CSV file
final_result.write.csv("Combined(1).csv", header=True)

# Download the CSV file to the system
from google.colab import files
files.download("Combined(1).csv")